In [145]:
qasm_one_param = """OPENQASM 2.0;
include "qelib1.inc";
gate ryy(param0) q0,q1 { rx(pi/2) q0; rx(pi/2) q1; cx q0,q1; rz(param0) q1; cx q0,q1; rx(-pi/2) q0; rx(-pi/2) q1; }
qreg q[2];
ryy(2.0425171585294746) q[1],q[0];
"""

qasm_one_param_one_gate = """OPENQASM 2.0;
include "qelib1.inc";
gate custom(param0,param1) q0 { rx(param1) q0; rz(param0) q0; }
qreg q[2];
custom(3.14,2000) q[0];
"""

qasm_two_param = """OPENQASM 2.0;
include "qelib1.inc";
gate xx_minus_yy(param0,param1) q0,q1 { rz(-1.0*param1) q1; ry(param0/2) q0; }
qreg q[2];
xx_minus_yy(2.00367210595874,5.07865952845335) q[0],q[1];
"""

qasm_three_qubit = """OPENQASM 2.0;
include "qelib1.inc";
gate ryy(param0) q0,q1,q2 { rx(pi/2) q0; rx(pi/2) q1; cx q0,q2; rz(param0) q1; cx q0,q1; rx(-pi/2) q2; rx(-pi/2) q1; }
qreg q[3];
ryy(2.0425171585294746) q[1],q[0],q[2];
"""

qasm_multi_gate = """OPENQASM 2.0;
include "qelib1.inc";
gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }
gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }
qreg q[2];
ecr q[0],q[1];
"""

qasm_str = qasm_three_qubit

In [158]:
import re


def format_qasm_string(qasm_string, skip_pattern):
    lines = qasm_string.split("\n")
    formatted_lines = []

    for line in lines:
        line = line.strip()  # Strip leading and trailing whitespace
        if skip_pattern.match(line):
            # If the line matches the gate definition pattern, add it as is
            formatted_lines.append(line)
        else:
            # Otherwise, split it at semicolons and add each part as a separate line
            parts = re.split(";[ ]*", line)
            parts = [part + ";" for part in parts if part]  # Remove empty parts
            formatted_lines.extend(parts)

    return "\n".join(formatted_lines)


def convert_qasm(qasm_string):
    # Define regular expression patterns
    gate_definition_pattern = re.compile(
        r"gate ([a-zA-Z0-9_]+)\((.*?)\) ((q[0-9]+,)*q[0-9]+) {(.*?)}"
    )

    # Find gate definition and extract its components
    gate_definition_match = gate_definition_pattern.search(qasm_string)
    if gate_definition_match:
        gate_name, params, qubits, _, gate_body = gate_definition_match.groups()
        params_list = [param.strip() for param in params.split(",")]

        qubits = [qubit.strip() for qubit in qubits.split(",")]

        gate_usage_pattern = re.compile(
            r"({})\((.*?)\) ((q\[([0-9]+)\],)*(q\[([0-9]+)\]));".format(gate_name)
        )

    # Replace parameters with their values in gate body
    gate_usage_match = gate_usage_pattern.search(qasm_string)
    while gate_usage_match:
        groups = gate_usage_match.groups()
        gate_call, param_values, qubits_usage = groups[0], groups[1], groups[2]
        param_values_list = [value.strip() for value in param_values.split(",")]
        expanded_gate_body = gate_body

        qubits_usage = [qubit.strip() for qubit in re.findall(r"q\[\d+\]", qubits_usage)]

        for param, value in zip(params_list, param_values_list):
            expanded_gate_body = expanded_gate_body.replace(param, value)

        for qubit, qubit_usage in zip(qubits, qubits_usage):
            expanded_gate_body = expanded_gate_body.replace(qubit, qubit_usage)

        # Replace gate usage with the expanded gate body in the input string
        qasm_string = qasm_string.replace(gate_usage_match.group(0), expanded_gate_body + ";")

        # Search for the next gate usage
        gate_usage_match = gate_usage_pattern.search(qasm_string)

    # Remove double semicolons
    qasm_string = format_qasm_string(qasm_string, gate_definition_pattern)

    return qasm_string

In [161]:
# qasm_one_param_one_gate
# qasm_two_param
# qasm_three_qubit
print(convert_qasm(qasm_one_param_one_gate))

OPENQASM 2.0;
include "qelib1.inc";
gate custom(param0,param1) q0 { rx(param1) q0; rz(param0) q0; }
qreg q[2];
rx(2000) q[0];
rz(3.14) q[0];
